# Import libraries and model

In [67]:
from os import listdir
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gc
import gensim.downloader as api

import torch
import torch.nn as nn

from math import ceil
import torch.optim as optim

nltk.download('stopwords')
nltk.download('wordnet')

# Load GloVe model with Gensim's API - Twitter specific embedding
embeddings_model = api.load("glove-twitter-200")  # 200-dimensional GloVe embeddings

#To check that T4 GPU is connected
#!nvidia-smi

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\panda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [68]:
# %pip install "modin[all]"
# import modin.pandas as pd

In [75]:
import pandas as pd

In [70]:
# %pip install swifter
# import swifter

In [76]:
import swifter

In [72]:
import os
os.environ["MODIN_ENGINE"] = "dask"  # Options: "ray" or "dask"
import modin.pandas as pd

In [73]:
swifter

<module 'swifter' from 'c:\\Users\\panda\\miniconda3\\envs\\ipx_torch\\lib\\site-packages\\swifter\\__init__.py'>

# Data preprocessing

In [77]:
# Read all training files and concatenate them into one dataframe

#import os
#print(os.getcwd())

li = []
i = 0
for filename in listdir("train_tweets"):
    if filename != '.ipynb_checkpoints':
        print(filename)
        df = pd.read_csv("train_tweets/" + filename)
        # df.drop(columns=['Timestamp'], inplace=True)
        # drop unused column(s)
        df['MatchID'] = str(i)
        df['ID'] = str(i)+ '_' + df['PeriodID'].astype(str)
        # makes sure that the match IDs are ordered from 0,1,2... with no missing values
        i+=1
        li.append(df)
df = pd.concat(li, ignore_index=True)
#print(len(df))
df

ArgentinaBelgium72.csv
ArgentinaGermanyFinal77.csv
AustraliaNetherlands29.csv
AustraliaSpain34.csv
BelgiumSouthKorea59.csv
CameroonBrazil36.csv
FranceGermany70.csv
FranceNigeria66.csv
GermanyAlgeria67.csv
GermanyBrazil74.csv
GermanyUSA57.csv
HondurasSwitzerland54.csv
MexicoCroatia37.csv
NetherlandsChile35.csv
PortugalGhana58.csv
USASlovenia2010.csv


,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1404575400000,RT @2014WorIdCup: Argentina vs Belgium\n\nWho ...
1,0_0,0,0,0,1404575400000,@elijahman_ time to focus on Belgium winning t...
2,0_0,0,0,0,1404575400000,RT @FIFAWorldCup: GLOBAL STADIUM: #Joinin with...
3,0_0,0,0,0,1404575400000,RT @CatholicNewsSvc: #PopeFrancis. Uh-oh. Arge...
4,0_0,0,0,0,1404575400000,RT @soccerdotcom: If he scores vs #BEL we'll a...
...,...,...,...,...,...,...
5056045,15_129,15,129,0,1276876799000,RT @nytimes FIFA World Cup -- Final Score: U.S...
5056046,15_129,15,129,0,1276876799000,Ugh!!! should've been 3-2 USA! #worldcup
5056047,15_129,15,129,0,1276876799000,RT @jaclynkeough: Ha! RT @someecards I'd rathe...
5056048,15_129,15,129,0,1276876799000,RT @gustavaulia: So many surprises at worldcup...


In [78]:
del li

In [79]:
gc.collect()

1709

In [80]:
# Preprocessing of tweet
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [81]:
# Apply preprocessing to each tweet
df['Tweet'] = df['Tweet'].swifter.apply(preprocess_text)
df

Pandas Apply:   0%|          | 0/5056050 [00:00<?, ?it/s]

,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1404575400000,rt woridcup argentina v belgium win httptcoleu...
1,0_0,0,0,0,1404575400000,elijahman_ time focus belgium winning world cup
2,0_0,0,0,0,1404575400000,rt fifaworldcup global stadium joinin worldcup...
3,0_0,0,0,0,1404575400000,rt catholicnewssvc popefrancis uhoh argentina ...
4,0_0,0,0,0,1404575400000,rt soccerdotcom score v bel well award messisi...
...,...,...,...,...,...,...
5056045,15_129,15,129,0,1276876799000,rt nytimes fifa world cup final score u sloven...
5056046,15_129,15,129,0,1276876799000,ugh shouldve usa worldcup
5056047,15_129,15,129,0,1276876799000,rt jaclynkeough ha rt someecards id rather die...
5056048,15_129,15,129,0,1276876799000,rt gustavaulia many surprise worldcup timewoww...


# Tweet Embeddings

In [83]:
# Get vector tweet embeddings
# TODOOOOOOOOOOOOOOOO maybe instead of avg word embedding for each tweet can get sentence
#   embeddings to retain more information
#   -> can try more complex functions here
#   -> avg embedding of each word for a tweet is fine for now, maybe works well enough

# Function to compute the average word vector for a tweet
def get_avg_embedding(tweet, model, vector_size=200):
    words = tweet.split()  # Tokenize by whitespace
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:  # If no words in the tweet are in the vocabulary, return a zero vector
        return np.zeros(vector_size)
    return np.sum(word_vectors, axis=0)

In [84]:
# Crashes after using all available RAM :( on google colab
# 

# obtain vector tweet embeddings
vector_size = 200  # Adjust based on the chosen GloVe model
# tweet_vectors = np.vstack([get_avg_embedding(tweet, embeddings_model, vector_size) for tweet in df['Tweet']])
# tweet_df = pd.DataFrame(tweet_vectors)
# tweet_df.head()

In [85]:
from functools import partial

f = partial(get_avg_embedding, model=embeddings_model,vector_size=vector_size)
f.__name__ = "paritla"
f.__module__ = get_avg_embedding.__module__

df["tweet_vector"] = df["Tweet"].swifter.apply(f)

Pandas Apply:   0%|          | 0/5056050 [00:00<?, ?it/s]

In [86]:
# Attach the vectors into the original dataframe
# df = pd.concat([df, tweet_df], axis=1)

# Drop the columns that are not useful anymore
# no need for Tweet column since we have its corresponding vector embedding
df.drop(columns=['Tweet'], inplace=True)



In [87]:
df.head()

,ID,MatchID,PeriodID,EventType,Timestamp,tweet_vector
0,0_0,0,0,0,1404575400000,"[1.8516128, 2.6584997, -0.28650498, 0.00972297..."
1,0_0,0,0,0,1404575400000,"[0.68378395, 1.078068, 0.556275, -1.0781101, -..."
2,0_0,0,0,0,1404575400000,"[-0.43702668, 2.7180884, 0.44844595, -2.479747..."
3,0_0,0,0,0,1404575400000,"[1.81652, 2.24445, -0.59934795, -0.66496515, -..."
4,0_0,0,0,0,1404575400000,"[2.5087266, 6.0086, -0.40718204, -0.7157027, 0..."


In [88]:
# by now should have df with columns: ID, match id, period id, Event Type, tweet_vector. Tweet_vector is just 200 columns
df

,ID,MatchID,PeriodID,EventType,Timestamp,tweet_vector
0,0_0,0,0,0,1404575400000,"[1.8516128, 2.6584997, -0.28650498, 0.00972297..."
1,0_0,0,0,0,1404575400000,"[0.68378395, 1.078068, 0.556275, -1.0781101, -..."
2,0_0,0,0,0,1404575400000,"[-0.43702668, 2.7180884, 0.44844595, -2.479747..."
3,0_0,0,0,0,1404575400000,"[1.81652, 2.24445, -0.59934795, -0.66496515, -..."
4,0_0,0,0,0,1404575400000,"[2.5087266, 6.0086, -0.40718204, -0.7157027, 0..."
...,...,...,...,...,...,...
5056045,15_129,15,129,0,1276876799000,"[2.8213418, 4.1725197, 3.30757, -3.5441713, 1...."
5056046,15_129,15,129,0,1276876799000,"[1.0414053, 0.5316, 0.98389703, 0.025289953, -..."
5056047,15_129,15,129,0,1276876799000,"[0.3141033, 4.4534097, 0.09538996, -0.5942371,..."
5056048,15_129,15,129,0,1276876799000,"[0.8237283, 2.6634202, 1.17653, -0.04314803, 0..."


In [93]:
# Sort by MatchID, PeriodID, and Timestamp to maintain order
df = df.sort_values(by=['MatchID', 'PeriodID', 'Timestamp']).reset_index(drop=True)

# Helper function to assign chunks
def assign_chunks(group, n_chunks=50):
    chunk_size = len(group) / n_chunks
    return (np.floor(np.arange(len(group)) / chunk_size)).astype(int)

# Apply chunk assignment within each MatchID and PeriodID
df['chunk'] = df.groupby(['MatchID', 'PeriodID']).apply(
    lambda group: assign_chunks(group)
).explode().reset_index(drop=True)

In [99]:
# group the tweets into their corresponding periods to generate an average embedding vector for each period
# so there are no duplicate period id rows per match
# decreases size of data + makes it easier to fit into LSTM model
df = df.groupby(['MatchID', 'PeriodID', 'ID','chunk']).sum().reset_index()
df.drop(columns=['ID'], inplace=True) 
df['MatchID'] = df['MatchID'].astype(int)
df['PeriodID'] = df['PeriodID'].astype(int)
# need to convert to int before sorting
df.sort_values(by=['MatchID', 'PeriodID', 'chunk'], inplace=True)
df.reset_index(drop=True, inplace=True)


In [100]:
df.loc[df["EventType"] != 0, "EventType"] = 1
df

,MatchID,PeriodID,chunk,EventType,Timestamp,tweet_vector
0,0,0,0,0,66015043813000,"[68.4425, 128.26672, 1.1481682, -29.883526, -4..."
1,0,0,1,0,66015043879000,"[68.88814, 145.43016, 10.056828, -33.073154, 5..."
2,0,0,2,0,64610468540000,"[71.49027, 131.55573, -3.6766365, -30.79529, -..."
3,0,0,3,0,66015043995000,"[55.109142, 128.37497, 7.5088744, -56.047688, ..."
4,0,0,4,0,64610468643000,"[73.1956, 143.64096, 20.510548, -56.184666, 7...."
...,...,...,...,...,...,...
106845,15,129,45,0,22983782302000,"[30.867702, 38.815422, 20.101866, -27.133873, ..."
106846,15,129,46,0,21706905525000,"[25.60558, 34.522087, 18.445267, -26.69125, -4..."
106847,15,129,47,0,22983782344000,"[27.194887, 41.0021, 22.521793, -24.38177, 1.9..."
106848,15,129,48,0,21706905570000,"[11.065972, 33.074814, 15.769472, -14.383462, ..."


In [101]:
df['tweet_vector'] = df['tweet_vector'].swifter.apply(lambda v: v / np.linalg.norm(v))

Pandas Apply:   0%|          | 0/106850 [00:00<?, ?it/s]

In [108]:
df_agg = (
    df.groupby(['MatchID', 'PeriodID'])
    .agg(
        period_matrix=('tweet_vector', lambda x: np.stack(x.to_numpy())),  # Stack the mean vectors
        mean_event_type=('EventType', 'mean')  # Compute the mean event type
    )
    .reset_index()
)

In [ ]:
df = df_agg

In [121]:
df.rename(columns={'period_matrix': 'tweet_vector', 'mean_event_type':'EventType'}, inplace=True)

In [122]:
df

,MatchID,PeriodID,tweet_vector,EventType
0,0,0,"[[0.03765871748328209, 0.07057545334100723, 0....",0.0
1,0,1,"[[0.025599336251616478, 0.0652989074587822, 0....",0.0
2,0,2,"[[0.029670257, 0.07340531, 0.006418672, -0.033...",0.0
3,0,3,"[[0.030907733365893364, 0.06012047082185745, 0...",0.0
4,0,4,"[[0.02883375, 0.07121538, 0.018995555, -0.0262...",0.0
...,...,...,...,...
2132,15,125,"[[0.034469195, 0.06374718, 0.044963595, -0.047...",1.0
2133,15,126,"[[0.044445183, 0.07559718, 0.03417508, -0.0509...",1.0
2134,15,127,"[[0.04381542, 0.07115657, 0.0370309, -0.053856...",1.0
2135,15,128,"[[0.0412188358604908, 0.07847695052623749, 0.0...",1.0


# Separate Train and Test data

In [123]:
# train on of the first 13 of 16 matches (16*0.8=12.8~=13)
# and the test data would be the last 3 matches. 
# Before submitting on Kaggle we should train on full dataset, so al 16 matches
train_percentage = 0.8
unique_match_ids = df['MatchID'].unique()
print(unique_match_ids)
num_matches_training = int(ceil(len(unique_match_ids)*train_percentage))
print(num_matches_training)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
13


In [124]:
target_match_id = num_matches_training
# target_match_id is first match id that will appear in test set
# all matches from target_match_id and after will be in test test
print(target_match_id)

13


In [88]:

#df2 = df['MatchID'] == 15
#df2

In [125]:
# row_index is first row with match id target_match_id
# row_index is then the first row of the matches that will go to the test


row_index = (df['MatchID'] == target_match_id).idxmax()
#row_index = df[df['MatchID'] == target_match_id].first_valid_index()
df_X_train = df[:row_index].copy()
df_X_test = df[row_index:].copy()


In [126]:
df_y_train = df_X_train['EventType']
df_y_test = df_X_test['EventType']

In [127]:
df_y_train

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1742    1.0
1743    1.0
1744    1.0
1745    1.0
1746    1.0
Name: EventType, Length: 1747, dtype: float64

In [128]:
df_y_test.reset_index(drop=True, inplace=True)
df_y_test

0      0.0
1      0.0
2      1.0
3      1.0
4      1.0
      ... 
385    1.0
386    1.0
387    1.0
388    1.0
389    0.0
Name: EventType, Length: 390, dtype: float64

In [129]:
df_X_train.drop(['EventType'], axis=1, inplace=True)
df_X_test.drop(['EventType'], axis=1, inplace=True)

In [130]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[[0.03765871748328209, 0.07057545334100723, 0...."
1,0,1,"[[0.025599336251616478, 0.0652989074587822, 0...."
2,0,2,"[[0.029670257, 0.07340531, 0.006418672, -0.033..."
3,0,3,"[[0.030907733365893364, 0.06012047082185745, 0..."
4,0,4,"[[0.02883375, 0.07121538, 0.018995555, -0.0262..."
...,...,...,...
1742,12,125,"[[0.0032179116, 0.08141216, 0.020280385, -0.05..."
1743,12,126,"[[0.011866402, 0.07410982, 0.019072963, -0.054..."
1744,12,127,"[[0.013964032, 0.07138574, 0.017871283, -0.060..."
1745,12,128,"[[0.011809887, 0.07534478, 0.016351322, -0.054..."


In [131]:
df_X_test.reset_index(drop=True, inplace=True)
df_X_test

,MatchID,PeriodID,tweet_vector
0,13,0,"[[0.04355798, 0.07092691, 0.0026004277, -0.025..."
1,13,1,"[[0.04902947, 0.06633132, -0.00224172, -0.0245..."
2,13,2,"[[0.038465895, 0.08396831, -0.016891256, -0.02..."
3,13,3,"[[0.04429667, 0.06179851, -0.0017939284, -0.02..."
4,13,4,"[[0.037068717, 0.06616976, -0.0033581697, -0.0..."
...,...,...,...
385,15,125,"[[0.034469195, 0.06374718, 0.044963595, -0.047..."
386,15,126,"[[0.044445183, 0.07559718, 0.03417508, -0.0509..."
387,15,127,"[[0.04381542, 0.07115657, 0.0370309, -0.053856..."
388,15,128,"[[0.0412188358604908, 0.07847695052623749, 0.0..."


In [96]:
# now df_X_train and df_X_test should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200 columns
# df_y_train and df_y_test should have 1 column, EventType
# the matchids are grouped together so all the rows of the same
# match ids are grouped next to each other, and the periodID are ordered chronologically.

In [97]:
# not working on my machine so I keep DF

# now we have df_X_train, df_X_test, df_y_train, df_y_test
# we no longer need df so we should free up the memory
# del df  # remove reference to the original DataFrame
# gc.collect()  # force garbage collection to free up memory

In [132]:
max_periods = df_X_train.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods
# as we can see not every match has the same number of periods!

,MatchID,PeriodID
0,0,129
1,1,179
2,2,96
3,3,129
4,4,129
5,5,129
6,6,129
7,7,129
8,8,169
9,9,129


In [133]:
max_periods = df_X_test.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods

,MatchID,PeriodID
0,13,129
1,14,129
2,15,129


# Format data for PyTorch LSTM

In [100]:
# input tensor for a PyTorch LSTM should have the shape of (when setting batch_first=True)
# (batch_size, seq_len, num_features) when using the batch_first=True parameter
# batch_size is number of sequences processed at once

# TRY WITHOUT SLIDING WINDOW APPROACH
#    which would mean batch size = number of matches
#    much easier to format for LSTM as 3D tensor
#    dimension of 3D tensor with batch_first=True:(batch_size = num_matches, seq_len = num_periods, num _features = 200)
#    (match_id, period_id, num_features=200)
#     not every match has the same number of periods!, so seq_len can vary between different matches
#     fix: will have to pad with zeroes
# we want tensor[match_id][period_id] to return list len 200 of corresponding tweet vector


In [136]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[[0.03765871748328209, 0.07057545334100723, 0...."
1,0,1,"[[0.025599336251616478, 0.0652989074587822, 0...."
2,0,2,"[[0.029670257, 0.07340531, 0.006418672, -0.033..."
3,0,3,"[[0.030907733365893364, 0.06012047082185745, 0..."
4,0,4,"[[0.02883375, 0.07121538, 0.018995555, -0.0262..."
...,...,...,...
1742,12,125,"[[0.0032179116, 0.08141216, 0.020280385, -0.05..."
1743,12,126,"[[0.011866402, 0.07410982, 0.019072963, -0.054..."
1744,12,127,"[[0.013964032, 0.07138574, 0.017871283, -0.060..."
1745,12,128,"[[0.011809887, 0.07534478, 0.016351322, -0.054..."


In [137]:
# modified for array tweet_vector column
def convert_df_to_3D_tensor(df_X, df_y):
    # df_X should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200x1 array
    # rows with same matchID should be grouped together (adjacent rows)
    # df_y should have one column (the EventType)
    # returns tensor_X numpy array already padded! shape: (num_matches, max_num_periods, num _features = 200)
    # and tensor_y of shape: (num_matches, max_num_periods)
    num_matches = len(df_X['MatchID'].unique())
    max_periods = df_X.groupby('MatchID')['PeriodID'].max().reset_index()
    total_max_period = max_periods['PeriodID'].max()
    #total_max_period is max seq len

    tensor_X = np.zeros((len(df_X), 50, 200))

    tensor_y = np.zeros((len(df_y,)))
    print(tensor_X.shape)
    print(tensor_y.shape)
    
    for row_index, row in df_X.iterrows():
        i = row_index        
        features = row['tweet_vector']  # Skip MatchID and PeriodID
        tensor_X[i, :, :] = features
        tensor_y[i] = df_y[row_index]
        
    return tensor_X, tensor_y


X_train_tensor, y_train_tensor = convert_df_to_3D_tensor(df_X_train, df_y_train)
# X_train_tensor[match_id][period_id] to return list len 200 of corresponding tweet vector
# y_train_tensor[match_id][period_id] to return corresponding EventType (1 or 0)
# match_id index starts at 0 even if first match in df doesnt have match id 0
#X_train_tensor[12][175]
#X_train_tensor[12][179]
#X_train_tensor[2][129]


(1747, 50, 200)
(1747,)


In [102]:
#print(X_train_tensor[0][3])
#print(y_train_tensor[0][3])

In [138]:
# SCALE DATA? minmaxscaler for example!
# SCALING MIGHT BE UNNECESSARY SINCE OUTPUT OF GLOVE TWEET 200 IS ALREADY SCALED BETWEEN -1 AND 1
#scaler = MinMaxScaler()
#tensor = scaler.fit_transform(tensor)

# CONVERT TO PYTORCH TENSOR
X_train_tensor = torch.tensor(X_train_tensor, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
# X_train_tensor, y_train_tensor are now pytorch tensors

torch.Size([1747, 50, 200])
torch.Size([1747])


# LSTM Model

In [141]:
# TODO VERIFY ITS CORRECT + MAKE MORE SOPHISTICATED
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        out = out[:,-1,:]
        out = self.sigmoid(out) # applying sigmoid to convert to probabilities
        return out.squeeze(-1)

#TODOOOOOOOOOO torch.nn.utils.rnn.pack_padded_sequence. This allows the model to ignore the padded values during computation.

In [148]:
batch_size = 64
batch_idxs = list(range(0,X_train_tensor.shape[0],batch_size))
batched_data = []
for idx in batch_idxs:
    batched_data.append((X_train_tensor[idx:idx+batch_size], y_train_tensor[idx:idx+batch_size]))

# Train model

In [ ]:
hidden_size = 500 # can tune
num_layers = 4 # can tune
dropout_rate = 0.2 # can tune
num_epochs = 500 # can tune
lr = 0.001 # can tune

model = LSTMModel(input_size=200, hidden_size=hidden_size, num_layers=num_layers, dropout_rate=dropout_rate)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss() # great for binary classification
#print(f"Shape of X_train_tensor: {X_train_tensor.shape}")
model.train()
for epoch in range(num_epochs):
    for train, label in batched_data:
        optimizer.zero_grad()
        outputs = model(train)
        #print(f"shape of outputs: {outputs.shape}")
        
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")

print("Model is trained! (on training data)")

Epoch [0/500], Loss: 0.6578
Epoch [10/500], Loss: 0.6091
Epoch [20/500], Loss: 0.6051
Epoch [30/500], Loss: 0.6027
Epoch [40/500], Loss: 0.6016


# Evaluate on test data

In [167]:
# convert df_X_test and df_y_test to correct format/dimensions
X_test_tensor, y_test_tensor = convert_df_to_3D_tensor(df_X_test, df_y_test)
# CONVERT TO PYTORCH TENSOR
X_test_tensor = torch.tensor(X_test_tensor, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

(3, 130, 200)
(3, 130)


In [117]:
#print(X_test_tensor[2][129])
#print(y_test_tensor[2][129])

In [168]:




model.eval()

with torch.no_grad():
    predictions = model(X_test_tensor)

# predictions have values between 0 and 1 because forward pass of LSTM contains sigmoid at output
#print(predictions)

predicted_classes = (predictions > 0.5).float() # 0.5 is threshold
#this converts to same dimensional array of True or false, and .float() converts True to 1 and False to 0

#print(predicted_classes)
 

In [2]:
predicted_classes

NameError: name 'predicted_classes' is not defined

In [119]:
#print(predictions.shape)

In [172]:
# performance metrics
from random import randint

loss = criterion(predictions, y_test_tensor) # use predictions for loss calculation

print(f"Binary Cross-Entropy Loss: {loss.item():.4f}")

def accuracy(y_true, y_pred):
    if y_true.dtype != y_pred.dtype or y_true.shape != y_pred.shape:
        raise ValueError(f"Inputs do not have same type or shape!")
    y_rand = torch.randint(0,2,y_true.shape)
    correct_predictions = (y_true == y_pred).sum().item()
    total_predictions = y_true.numel()
    accuracy = correct_predictions / total_predictions * 100
    return accuracy
accuracy = accuracy(y_test_tensor, predicted_classes)


print(f"Accuracy: {accuracy:.4f}")

#print(y_test_tensor.shape)
#print(predicted_classes.shape)


# Visualization of Actual vs Predicted Classes
# import matplotlib.pyplot as plt
# TODO COULD USE PLT TO VISUALIZE?

Binary Cross-Entropy Loss: 1.3613
Accuracy: 68.2051


# For Kaggle Submission

In [111]:
# RETRAIN MODEL ON ENTIRE TRAINING DATA AND EVALUATE EVAL TWEETS




df_X = pd.concat([df_X_train, df_X_test], ignore_index=True)
df_y = pd.concat([df_y_train, df_y_test], ignore_index=True)




In [112]:
df_X

,MatchID,PeriodID,tweet_vector
0,0,0,"[3098.4608585517853, 6436.185963880271, 363.53..."
1,0,1,"[2493.7816506992094, 5349.19354169257, 284.537..."
2,0,2,"[1972.8186, 4104.4824, 323.55396, -1203.197, -..."
3,0,3,"[2432.1588252037764, 5373.947940472513, 723.78..."
4,0,4,"[2607.9675, 5692.466, 748.0113, -1843.9877, -3..."
...,...,...,...
2132,15,125,"[1768.0021, 3085.3567, 1567.4435, -1976.7213, ..."
2133,15,126,"[1547.5847, 2672.128, 1288.0339, -1692.0323, 4..."
2134,15,127,"[1389.9213, 2417.8992, 1197.792, -1539.6497, 3..."
2135,15,128,"[1233.3035642304458, 2106.0893550515175, 1090...."


In [113]:
df_y

0       0
1       0
2       0
3       0
4       0
       ..
2132    1
2133    1
2134    1
2135    1
2136    0
Name: EventType, Length: 2137, dtype: int64

In [114]:
# convert df_X_test and df_y_test to correct format/dimensions
X_tensor, y_tensor = convert_df_to_3D_tensor(df_X, df_y)
# CONVERT TO PYTORCH TENSOR
X_tensor = torch.tensor(X_tensor, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor, dtype=torch.float32)

(16, 180, 200)
(16, 180)


In [115]:
# NOTES
# HOW TO MAKE SURE THAT we:
# 1. DO NOT ignore the order of the tweets -> (LSTM)
# 2. treat each time period as RELATED to the football match they belong to -> treat each match as a sequence, train LSTM on every sequence
#                      since pytorch tensor expects multiple sequences (batches)



# for LSTM: Each input sequence should consist of tweets from a specific match, ordered by Period ID.
#   tweets of different matches are unrelated, but tweets of a same match are related sequentially (chronologically)
#   structure training data such that tweets are grouped by match id, and ordered by period id